<a href="https://colab.research.google.com/github/ecsquare/Covid-19/blob/master/Short_text_based_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Topic modeling for short texts is an increasingly impor-tant task due to the prevalence of short texts on the Web.Compared to normal documents, short texts lack of wordfrequency and context information, causing severe sparsityproblems for conventional topic models


In [0]:
%matplotlib inline
import pickle
from pprint import pprint
import random
import warnings
import time

# numpy, pandas, matplotlib and regular expressions 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# tqdm
from tqdm import tqdm

# spacy
import spacy
from spacy.lang.en import English
import en_core_web_sm

# gensim
import gensim
from gensim import corpora
from gensim.models import CoherenceModel

# nltk
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import words
from nltk.stem.wordnet import WordNetLemmatizer

# pyLDAvis
import pyLDAvis
import pyLDAvis.gensim


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
cd /content/gdrive/My\ Drive/Colab\ Notebooks/vegan_tweets_2020-05-/

In [0]:
ls

preparing Dataset


In [4]:
import pandas as pd
df1 = pd.read_json (r'vegan_2020-05-01.json',lines=True)
df2 = pd.read_json(r'vegan_2020-05-02 2.json', lines=True)
df3 = pd.read_json(r'vegan_2020-05-03.json', lines=True)
df4 = pd.read_json(r'vegan_2020-05-04.json', lines=True)
df5 = pd.read_json(r'vegan_2020-05-05.json', lines=True)
df6 = pd.read_json(r'vegan_2020-05-06.json', lines=True)
df7 = pd.read_json(r'vegan_2020-05-07 2.json', lines=True)

frames = [df1, df2, df3,df4,df5,df6,df7]
df=pd.concat(frames)
df=df[['id','tweet','hashtags']]
df.columns
df.head(10)

,id,tweet,hashtags
0,1256372793752653825,how bout y’all compare shit that actually make...,[]
1,1256372790497644545,( http://HappyCow.net is an excellent search...,[]
2,1256372760638566401,Whew RT @_underhereye: Brussels Sprouts cooked...,[]
3,1256372759619350528,vegan labouyi (plantiain porridge). one of my ...,[]
4,1256372716141232129,Sounds a bit like a veterinarian vegan sort of...,[]
5,1256372705940705280,"no girl has all three: synesthesia, internal m...",[]
6,1256372679784960006,I definitely need to do my hair in NYC again soon,[]
7,1256372662210879489,how u goin vegan but still beefin w me again?!...,[#newprofilepic]
8,1256372646117117952,Happy Friday! I had some @traderjoes herb pizz...,[#vegan]
9,1256372631130894336,Which do you prefer? Gummies or capsules? ⭐️⭐️...,"[#hairtamin, #vegan, #vegeterian, #hairvitamins]"


Data cleaning

In [0]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation

In [0]:

def clean_tweets(text):
    
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
def cleanhashtag(text):
    text = clean_tweets(text)
    text= " ".join(text.split('#'))
    return text


In [0]:
stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This','me','my','it','you','your','they']

In [0]:
def clean_data(text, remove_stop_words=True, stem_words=True):
    # Clean the text, with the option to remove stop_words and to stem words.
   
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"http", " ", text)
    text = re.sub(r"https", " ", text)
    text = re.sub(r"www", " ", text)

    text = re.sub(r".com", " ", text)
    text = re.sub(r".fr", " ", text)
    text = re.sub(r".org", " ", text)
    text = re.sub(r"img", " ", text)


    #remove vegan 
    text = re.sub(r"vegan", " ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
  
    return(text)



In [142]:
df['tweet'] = df['tweet'].apply(lambda x: clean_data(x))
df['hashtags']=  df['hashtags'].apply(str)
df['hashtags']=  df['hashtags'].apply(lambda x: clean_data(x))
df.head(10)

,id,tweet,hashtags,Tweet Length
0,1256372793752653825,how bout y all pare shit actual make sen like ...,,89
1,1256372790497644545,happycow net excel search tool vegetarian veg ...,,170
2,1256372760638566401,whew rt underherey brussel sprout cook in butt...,,92
3,1256372759619350528,labouyi plantiain porridg one favorit meal gro...,,88
4,1256372716141232129,sound bit like veterinarian sort thing,,38
5,1256372705940705280,no girl has all three synesthesia intern monologu,,49
6,1256372679784960006,i definit need do hair in nyc again soon,,40
7,1256372662210879489,how u goin still beefin w again newprofilep pi...,newprofilep,66
8,1256372646117117952,happi iday i had some traderjo herb pizza doug...,,243
9,1256372631130894336,do prefer gummi capsul hairtamin vegeterian ha...,hairtamin vegeterian hairvitamin,78


In [143]:
df["Tweet Length"]= df["tweet"].str.len() 
df.head(10)

,id,tweet,hashtags,Tweet Length
0,1256372793752653825,how bout y all pare shit actual make sen like ...,,89
1,1256372790497644545,happycow net excel search tool vegetarian veg ...,,166
2,1256372760638566401,whew rt underherey brussel sprout cook in butt...,,92
3,1256372759619350528,labouyi plantiain porridg one favorit meal gro...,,88
4,1256372716141232129,sound bit like veterinarian sort thing,,38
5,1256372705940705280,no girl has all three synesthesia intern monologu,,49
6,1256372679784960006,i definit need do hair in nyc again soon,,40
7,1256372662210879489,how u goin still beefin w again newprofilep pi...,newprofilep,66
8,1256372646117117952,happi iday i had some traderjo herb pizza doug...,,243
9,1256372631130894336,do prefer gummi capsul hairtamin vegeterian ha...,hairtamin vegeterian hairvitamin,78


In [0]:
df = df[df['Tweet Length'] >10]

In [0]:
import matplotlib.pyplot as plt

df['Tweet Length'].plot(kind='bar');

In [145]:
print("mean=", df['Tweet Length'].mean(), "max=", df['Tweet Length'].max(), "min=",df['Tweet Length'].min())

mean= 98.07973168039399 max= 472 min= 11


In [127]:
df.hashtags.value_counts()

                                                                                             9960
shopmycloset                                                                                   26
plantbasedparti                                                                                19
recip food cook delici cook recip                                                              13
travel armchairtravel                                                                          13
                                                                                             ... 
herb healthiswealth goodnew heal juic burdockroot black healthyfood alkalin plantba mucus       1
chee nightmaresaremadeofch go food                                                              1
chinesenewnetwork                                                                               1
glute ee dair ee kosher nongmo hike exerci bar gain camp gym                                    1
yelp                

**BiTerM** 

In [146]:
data = df['tweet'].values.tolist()
len(data)

11777

In [0]:
with open('TweetCorpus.txt', 'w') as f:
    for i in data:
        f.write("%s\n" % i)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(data).toarray()

Install biterm and pyLDavis

In [0]:
!pip install biterm


In [0]:
!pip install pyLDAvis

In [0]:
import numpy as np 
from biterm.utility import vec_to_biterms

vocab = np.array(vec.get_feature_names())
biterms = vec_to_biterms(X[:200])

In [0]:
biterms

In [0]:
from biterm.btm import oBTM

btm = oBTM(num_topics=20, V=vocab)

In [0]:
topics = btm.fit_transform(biterms, iterations=100)

In [0]:
# Enabling pyLDAvis
pyLDAvis.enable_notebook()

In [140]:
import pyLDAvis
print("\n\n Visualize Topics ..")

vis = pyLDAvis.prepare(btm.phi_wz.T, topics, np.count_nonzero(X[:200], axis=1), vocab, np.sum(X[:200], axis=0))
pyLDAvis.save_html(vis, 'testoutput_btm.html')
vis



 Visualize Topics ..


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14    -0.114261 -0.080592       1        1  7.198176
5     -0.029588 -0.092811       2        1  6.517283
19    -0.052286 -0.124774       3        1  6.367128
17    -0.037518  0.162006       4        1  6.131804
4      0.135289  0.074086       5        1  6.011187
0     -0.184171 -0.002143       6        1  5.929068
9     -0.089650 -0.005212       7        1  5.873545
2     -0.127066  0.239923       8        1  5.818379
18     0.040675  0.117864       9        1  5.512071
15    -0.120947  0.111434      10        1  5.446662
1     -0.106512 -0.073116      11        1  5.218175
11     0.031349 -0.136812      12        1  4.787263
10    -0.040969 -0.090434      13        1  4.585363
13     0.073466  0.095981      14        1  4.322394
3      0.156968  0.073858      15        1  4.140384
12    -0.015434 -0.164724      16        1  4.029856
6     -0.065417 -0.029988      17        1  3.506342
8      0.150155 -0.004695      18        1  3.447083
7      0.212956 -0.032791      19        1  2.587238
16     0.182959 -0.037058      20        1  2.570599, topic_info=          Term       Freq      Total Category  logprob  loglift
3438        at  12.000000  12.000000  Default  30.0000  30.0000
14636      pic  37.000000  37.000000  Default  29.0000  29.0000
20440     with  21.000000  21.000000  Default  28.0000  28.0000
19147  twitter  45.000000  45.000000  Default  27.0000  27.0000
9370      have  19.000000  19.000000  Default  26.0000  26.0000
...        ...        ...        ...      ...      ...      ...
20440     with   1.699908  21.180534  Topic20  -3.7582   1.1385
17394     some   1.378390   9.718771  Topic20  -3.9678   1.7079
19147  twitter   1.929563  45.618102  Topic20  -3.6315   0.4980
14636      pic   1.653977  37.341402  Topic20  -3.7856   0.5441
10221       in   1.332459  27.217908  Topic20  -4.0017   0.6442

[1126 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
12        16  0.903665          01
14         1  0.808579          02
23        16  0.903665          05
229       18  0.898240          12
230        2  0.950733         120
...      ...       ...         ...
21022     17  0.320347      youtub
21063      5  1.213275         yum
21113      5  1.213275  z5dyr573jd
21157     11  1.246084        zero
21203      3  0.975287  zlbivznjn2

[1500 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 6, 20, 18, 5, 1, 10, 3, 19, 16, 2, 12, 11, 14, 4, 13, 7, 9, 8, 17])

Methode 2: **GSDMM**

In [0]:

import random

class GibbsSamplingDMM(object):
	
	numDocuments = 0
	numWordsInCorpus = 0
	word2IdVocabulary = {}
	id2WordVocabulary = {}
	documents = []
	occurenceToIndexCount = []
	topicAssignments = []
	docTopicCount = []
	topicWordCount = []
	sumTopicWordCount = []
	multiPros = []
	betaSum=0.

	def __init__(self, paramters):
		super(GibbsSamplingDMM, self).__init__()
		self.corpus = paramters["corpus"]
		self.output = paramters["output"]
		self.ntopics = int(paramters["ntopics"])
		self.alpha = float(paramters["alpha"])
		self.beta = float(paramters["beta"])
		self.niters = int(paramters["niters"])
		self.twords = int(paramters["twords"])
		self.name = paramters["name"]

	def analyseCorpus(self):
		indexWord=0
		data = open(self.corpus).read().splitlines()[:50]
		for doc in data:
			document = []
			wordOccurenceToIndexInDocCount = {}
			wordOccurenceToIndexInDoc = []
			if doc.rstrip!=None:
				words = doc.rstrip().split()
				for word in words:

					if word in self.word2IdVocabulary:
						document.append(self.word2IdVocabulary[word])
					else:
						self.word2IdVocabulary[word]=indexWord
						self.id2WordVocabulary[indexWord]=word
						document.append(indexWord)
						indexWord+=1

					if word in wordOccurenceToIndexInDocCount:
						wordOccurenceToIndexInDocCount[word]+=1
					else:
						wordOccurenceToIndexInDocCount[word]=1

					wordOccurenceToIndexInDoc.append(wordOccurenceToIndexInDocCount[word])

				self.numWordsInCorpus+=len(document)
				self.numDocuments+=1
				self.documents.append(document)
				self.occurenceToIndexCount.append(wordOccurenceToIndexInDoc)

		self.betaSum=len(self.word2IdVocabulary)*self.beta


	def topicAssigmentInitialise(self):
		self.docTopicCount = [0 for x in range(self.ntopics)]
		self.sumTopicWordCount = [0 for x in range(self.ntopics)]

		for i in range(self.ntopics):
				self.topicWordCount.append([0 for x in range(len(self.word2IdVocabulary))])

		for i in range (self.numDocuments):
			topic = random.randint(0,self.ntopics-1)
			self.docTopicCount[topic]+=1
			
			for j in range (len(self.documents[i])):
				self.topicWordCount[topic][self.documents[i][j]]+=1
				self.sumTopicWordCount[topic]+=1

			self.topicAssignments.append(topic)

	def nextDiscrete(self,a):
		b = 0.

		for i in range(len(a)):
			b+=a[i]

		r = random.uniform(0.,1.)*b
		
		b=0.
		for i in range (len(a)):
			b+=a[i]
			if(b>r):
				return i
		return len(a)-1

	def sampleInSingleIteration(self,x):
		print ("iteration: "+str(x))
		for d in range(self.numDocuments):
			topic = self.topicAssignments[d]
			self.docTopicCount[topic]-=1
			docSize = len(self.documents[d])
			document = self.documents[d]

			for w in range(docSize):
				word = document[w]
				self.topicWordCount[topic][word]-=1
				self.sumTopicWordCount[topic]-=1
			

			for t in range(self.ntopics):
				self.multiPros[t] = self.docTopicCount[t]+self.alpha

				for w in range(docSize):
					word = document[w]
					self.multiPros[t] *= (self.topicWordCount[t][word]+self.beta+self.occurenceToIndexCount[d][w]-1)/(self.sumTopicWordCount[t]+w+self.betaSum)
				
					
			#print self.multiPros
			topic = self.nextDiscrete(self.multiPros)
			#print topic

			self.docTopicCount[topic]+=1

			for w in range(docSize):
				word = document[w]
				self.topicWordCount[topic][word]+=1
				self.sumTopicWordCount[topic]+=1

			self.topicAssignments[d] = topic

	def inference(self):
		self.multiPros = [0 for x in range(self.ntopics)]
		[self.sampleInSingleIteration(x) for x in range(self.niters)]

	def writeTopicAssignments(self):
		file = open(self.output+self.name+"topicAssignments.txt","w")
		#for i in range(self.numDocuments):
		[file.write(str(self.topicAssignments[i])+"\n") for i in range(self.numDocuments)]



	def writeTopTopicalWords(self):
		file = open(self.output+self.name+"topWords.txt","w") 
		for t in range(self.ntopics):
			wordCount = {w:self.topicWordCount[t][w] for w in range(len(self.word2IdVocabulary))}
			
			count =0
			string="Topic "+str(t)+": "
			
			for index in sorted(wordCount, key=wordCount.get, reverse=True):
				string += self.id2WordVocabulary[index]+" "
				count+=1
				if count>=self.twords:
					file.write(string+"\n") 
					# print string
					break
		file.close()

In [0]:
params = {"corpus":"TweetCorpus.txt", 
          "output": "Testout", 
          "ntopics": "20", 
          "alpha":"0.1",
          "beta":"0.1", 
          "niters":"300",
          "twords":"20",
          "name":""}

model = GibbsSamplingDMM(params)
model.analyseCorpus()
model.topicAssigmentInitialise()
model.inference()
	
print ("Writing Results")
model.writeTopTopicalWords()
model.writeTopicAssignments()

Method 3: **LDA ** 


Firstly, I am creating a dictionary from our data, then I am converting it into a bag-of-words corpus and then saving the dictionary and corpus for future use using pickle library

In [0]:

parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [0]:
text_data = []
for row in tqdm(data):
    tokens = tokenize(row)
    text_data.append(tokens)

In [0]:
print("Generating corpus and dictionary...")

dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [0]:
%%time
print("Training LDA model on train dataframe...")

NUM_TOPICS = 20
start_time = time.time()
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics=NUM_TOPICS, 
                                           random_state=89, 
                                           update_every=1,  
                                           id2word=dictionary, 
                                           passes=42, 
                                           alpha='auto', 
                                           per_word_topics=True)

ldamodel.save('model' + str(NUM_TOPICS) + '.gensim')
train_time = time.time() - start_time
print("Training Time --- %s seconds " % (round(train_time, 2)))

In [0]:
print("Allocated models of each topic...")

topics_train = []
topics = ldamodel.print_topics()
for topic in topics:
    pprint(topic)
    topics_train.append(topic)
    

with open("results_train.txt", 'w') as f:
    pprint(topics_train, stream=f)

In [0]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=text_data, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [176]:
# Visualizing our optimal model with 100 topics
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.save_html(vis, 'testoutput_LDA.html')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.304179  0.296623       1        1  51.687069
15    -0.223756 -0.129354       2        1   5.103079
17    -0.128300 -0.044577       3        1   4.143746
6     -0.158113 -0.170513       4        1   3.649071
5      0.078003  0.196445       5        1   3.460646
13    -0.118622 -0.040881       6        1   3.390307
10     0.082215  0.192579       7        1   2.783545
8      0.032316 -0.003978       8        1   2.407683
7      0.015718  0.001158       9        1   2.348744
16     0.064520  0.015482      10        1   2.331523
1      0.056112 -0.030295      11        1   2.185484
11    -0.055165 -0.116434      12        1   2.184935
12     0.020659 -0.036264      13        1   2.081486
19     0.107694  0.017146      14        1   1.964960
9      0.072713 -0.032572      15        1   1.881785
3      0.044627  0.035781      16        1   1.839280
4      0.055451 -0.069498      17        1   1.733575
18     0.121222 -0.032831      18        1   1.687580
14     0.131480 -0.017067      19        1   1.665505
0      0.105404 -0.030950      20        1   1.470001, topic_info=         Term         Freq        Total Category  logprob  loglift
52    twitter  2489.000000  2489.000000  Default  30.0000  30.0000
31          s  4604.000000  4604.000000  Default  29.0000  29.0000
7           i  7900.000000  7900.000000  Default  28.0000  28.0000
48        pic  1883.000000  1883.000000  Default  27.0000  27.0000
148        go  1306.000000  1306.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
1578  homemad    25.467995    94.190613  Topic20  -4.8156   2.9120
385        om    18.826614   810.585327  Topic20  -5.1177   0.4574
899   skincar    12.769567    61.045105  Topic20  -5.5060   2.6554
199       off    11.340919   192.215332  Topic20  -5.6246   1.3897
523         5    10.708985   159.560913  Topic20  -5.6819   1.5186

[1013 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
2089     17  0.966453       0
3145     18  0.947095      00
1242      2  0.965004      02
615       2  0.988763      05
3516      9  0.946754      07
...     ...       ...     ...
281      12  0.989668   youtu
557       4  0.292848  youtub
557      12  0.702835  youtub
1012      3  0.862850     yum
1012     20  0.104588     yum

[1511 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 16, 18, 7, 6, 14, 11, 9, 8, 17, 2, 12, 13, 20, 10, 4, 5, 19, 15, 1])